In [1]:
from pathlib import Path
from datetime import datetime
import helper
import yaml, pandas as pd, numpy as np, tqdm.auto as tqdm, shutil, subprocess


In [2]:
param_path = "/home/julienb/Documents/database_scripts/run_configs/run_config_v1.yaml"

In [3]:
param_path = Path(param_path)
params = yaml.safe_load(param_path.open("r"))
print(params)

{'variables': {'base_folder': '/home/julienb/Documents/database_scripts', 'this_run_config': 'test_runconfig_v1'}, 'runs': [{'script': 'polydat_file_to_events2.ipynb', 'imports': ['events_methods.py', 'helper.py'], 'environment': 'dbscripts', 'script_params': {'dat_path': '/home/julienb/Documents/database_scripts/analysis_test_data/poly_dat_files/Rats/Luisa/Rat101_0729_opto_01/Rat101_0729_opto_01.dat', 'task_path': '/home/julienb/Documents/database_scripts/analysis_test_data/poly_dat_files/Rats/Luisa/Rat101_0729_opto_01.xls', 'config_path': '/home/julienb/Documents/database_scripts/templates/poly_rat_model.yaml', 'dest_path': 'events.tsv'}, 'run_folder': '{base_folder}/runs/{this_run_config}/poly'}]}


In [4]:
handled_params = helper.replace_vals(params["runs"], params["variables"])
handled_params


[{'script': 'polydat_file_to_events2.ipynb',
  'imports': ['events_methods.py', 'helper.py'],
  'environment': 'dbscripts',
  'script_params': {'dat_path': '/home/julienb/Documents/database_scripts/analysis_test_data/poly_dat_files/Rats/Luisa/Rat101_0729_opto_01/Rat101_0729_opto_01.dat',
   'task_path': '/home/julienb/Documents/database_scripts/analysis_test_data/poly_dat_files/Rats/Luisa/Rat101_0729_opto_01.xls',
   'config_path': '/home/julienb/Documents/database_scripts/templates/poly_rat_model.yaml',
   'dest_path': 'events.tsv'},
  'run_folder': '/home/julienb/Documents/database_scripts/runs/test_runconfig_v1/poly'}]

In [5]:
run_df = pd.DataFrame(handled_params).assign(run_num=np.arange(len(handled_params)))
if run_df["run_folder"].duplicated().any():
    raise Exception("run folder must be different")
run_df
    

,script,imports,environment,script_params,run_folder,run_num
0,polydat_file_to_events2.ipynb,"[events_methods.py, helper.py]",dbscripts,{'dat_path': '/home/julienb/Documents/database...,/home/julienb/Documents/database_scripts/runs/...,0


In [6]:
infos = []
for _, row in tqdm.tqdm(run_df.iterrows(), total=len(run_df.index)):
    current_info = {}
    current_info["run_num"] = row["run_num"]
    current_info["script"] = row["script"]
    current_info["start_time"] = datetime.now()
    try:
        run_folder = Path(row["run_folder"]+".tmp")
        if run_folder.exists():
            shutil.rmtree(run_folder)
        dest_folder = Path(row["run_folder"])
        run_folder.mkdir(exist_ok=True, parents=True)
        if "imports" in row:
            if not isinstance(row["imports"], list):
                row["imports"] = [row["imports"]]
            for im in row["imports"]:
                shutil.copy(im, run_folder / im)
        # shutil.copy(row['script'], run_folder/row['script'])
        with (run_folder / "params.yaml").open("w") as f:
            yaml.dump(row["script_params"], f)
        # if "environment" in row:
        #     subprocess.run(f"conda run -n {row['environment']} papermill '{run_folder / row['script']}'", shell=True)
        # else:
        subprocess.run(f"conda run -n dbscripts papermill --cwd '{run_folder}' '{row['script']}' '{run_folder/row['script']}' ", shell=True, check=True)

        
    except Exception as e:
        current_info["sucess"] = False
        print(f"Error during run number {row['run_num']}")
        display(e)
    else:
        current_info["sucess"] = True
    try:
        subprocess.run(f'jupyter nbconvert --to html {run_folder/row["script"]}', shell=True)
    except Exception as e:
        print("Error while attempting to convert to notebook")
        display(e)
        current_info["html_sucess"] = False
    else:
        current_info["html_sucess"] = True
    if (run_folder / "__pycache__").exists():
        shutil.rmtree(run_folder / "__pycache__")
    if current_info["sucess"] & current_info["html_sucess"]:
        if dest_folder.exists():
            shutil.rmtree(dest_folder)
        shutil.move(run_folder, dest_folder)
    current_info["end_time"] = datetime.now()
    infos.append(current_info)
infos = pd.DataFrame(infos)
infos["duration"] = (infos["end_time"] - infos["start_time"]).dt.total_seconds()
infos=infos.drop(columns=["end_time"])
infos

  0%|          | 0/1 [00:00<?, ?it/s]

Input Notebook:  polydat_file_to_events2.ipynb
Output Notebook: /home/julienb/Documents/database_scripts/runs/test_runconfig_v1/poly.tmp/polydat_file_to_events2.ipynb
Working directory: /home/julienb/Documents/database_scripts/runs/test_runconfig_v1/poly.tmp

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]Executing notebook with kernel: python3

Executing: 100%|██████████| 19/19 [00:03<00:00,  5.51cell/s]

[NbConvertApp] Converting notebook /home/julienb/Documents/database_scripts/runs/test_runconfig_v1/poly.tmp/polydat_file_to_events2.ipynb to html
[NbConvertApp] Writing 2505899 bytes to /home/julienb/Documents/database_scripts/runs/test_runconfig_v1/poly.tmp/polydat_file_to_events2.html


,run_num,script,start_time,sucess,html_sucess,duration
0,0,polydat_file_to_events2.ipynb,2024-10-21 17:44:53.804541,True,True,5.131564
